In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving npr.csv to npr (4).csv
User uploaded file "npr.csv" with length 55511119 bytes


In [ ]:
# Preprocessing on npr dataset 
# www.npr.org
import pandas as pd
npr = pd.read_csv('npr (4).csv')
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [ ]:
len(npr)
print(npr.shape)


(11992, 1)


In [ ]:
npr['Article'][1]

'  Donald Trump has used Twitter  —   his preferred means of communication  —   to weigh in on a swath of foreign policy issues over the past few weeks. His comments give a glimpse into how his incoming administration will deal with pressing foreign matters  —   but also highlight how reactionary comments on social media can immediately spur international concern and attention. And his staff has indicated that taking to Twitter to air his concerns or, often, grievances, won’t end once he enters the Oval Office. On Wednesday, Trump blasted the U. S.’s abstention from the U. N. Security Council vote on Israeli settlements earlier this month. The tweets came just hours before Secretary of State John Kerry gave a speech defending the decision and calling the continued building of settlements on Palestinian territory in the West Bank a threat to the   solution in the region. Trump’s support for Israel and Prime Minister Benjamin Netanyahu  —   who has had a fraught relationship with Preside

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = cv.fit_transform(npr['Article'])
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [ ]:
#from sklearn.decomposition import LatentDirichletAllocation
#LDA = LatentDirichletAllocation(n_components=7,random_state=42)
from sklearn.decomposition import NMF
LDA = NMF(n_components=7,random_state=42)
LDA.fit(dtm)

# Three Steps Away...
### Grab the vocab.
### Grab the topic
### and finally grab the highest prob. word per topic



In [ ]:
len(cv.get_feature_names())

54777

In [ ]:
type(cv.get_feature_names())

list

In [ ]:
cv.get_feature_names()[5510]

'beret'

In [ ]:
import random

In [ ]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(cv.get_feature_names()[random_word_id])

lasserre
toasts
snarled
inefficiencies
dubious
misfolded
queried
spooked
goofball
strauss


### Showing Top Words Per Topic

In [ ]:
len(LDA.components_)

7

In [ ]:
LDA.components_.shape

(7, 54777)

In [ ]:
LDA.components_

In [ ]:
len(LDA.components_[0])

54777

In [ ]:
single_topic = LDA.components_[0]

In [ ]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([17319, 18349, 40552, ..., 26752, 36283, 28659])

In [ ]:
#lets understand the argsort in simple example
import numpy as np
arr = np.array([10,200,1])
arr.argsort()


array([2, 0, 1])

In [ ]:
# Word least representative of this topic
single_topic[18302]

0.13804750358531165

In [ ]:
# Word most representative of this topic
single_topic[42997]

0.0

In [ ]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([15008, 53211, 49459, 21073, 39848, 27439, 49183, 26752, 36283,
       28659])

In [ ]:
top_word_indices = single_topic.argsort()[-10:]

In [ ]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

don
way
time
going
really
know
think
just
people
like


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [ ]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['new', 'lot', 'make', 'things', 've', 'don', 'way', 'time', 'going', 'really', 'know', 'think', 'just', 'people', 'like']


THE TOP 15 WORDS FOR TOPIC #1
['presidential', 'republicans', 'comey', 'administration', 'obama', 'new', 'election', 'republican', 'white', 'donald', 'house', 'campaign', 'said', 'president', 'trump']


THE TOP 15 WORDS FOR TOPIC #2
['government', 'say', 'city', 'day', 'university', 'year', 'water', 'food', 'time', 'just', 'new', 'years', 'like', 'people', 'says']


THE TOP 15 WORDS FOR TOPIC #3
['according', 'reports', 'news', 'department', 'new', 'law', 'time', 'government', 'house', 'npr', 'court', 'told', 'president', 'police', 'said']


THE TOP 15 WORDS FOR TOPIC #4
['country', 'election', 'new', 'states', 'vote', 'party', 'obama', 'democratic', 'voters', 'percent', 'campaign', 'hillary', 'state', 'sanders', 'clinton']


THE TOP 15 WORDS FOR TOPIC #5
['new', 'percent', 'year', 'parents', 'private', 'kids', 'children', 'student',

### Attaching Discovered Topic Labels to Original Articles

In [ ]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [ ]:
dtm.shape

(11992, 54777)

In [ ]:
len(npr)

11992

In [ ]:
topic_results = LDA.transform(dtm)

In [ ]:
topic_results.shape

(11992, 7)

In [ ]:
topic_results[0]

array([0.05459945, 0.88996904, 0.00661222, 0.24766001, 0.18977488,
       0.01347329, 0.        ])

In [ ]:
topic_results[0].round(2)

array([0.05, 0.89, 0.01, 0.25, 0.19, 0.01, 0.  ])

In [ ]:
topic_results[0].argmax()

1

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [ ]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [ ]:
topic_results.argmax(axis=1)

array([1, 1, 1, ..., 3, 4, 2])

In [ ]:
npr['Topic'] = topic_results.argmax(axis=1)

In [ ]:
npr.head(20)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3
4,"From photography, illustration and video, to d...",5
5,I did not want to join yoga class. I hated tho...,0
6,With a who has publicly supported the debunk...,6
7,"I was standing by the airport exit, debating w...",2
8,"If movies were trying to be more realistic, pe...",2
9,"Eighteen years ago, on New Year’s Eve, David F...",2


##NON NEGATIVE MATRIX FACTORIZATION ALGORITHMS

In [ ]:
#https://scikit-learn.org/stable/modules/decomposition.html#nmf

import numpy as np
from sklearn.decomposition import NMF
X = np.array([[1, 1], [2, 1], [3, 1.2], [4, 1], [5, 0.8], [6, 1]])


In [ ]:
model = NMF(n_components=2, init='random', random_state=0)


In [ ]:
W = model.fit_transform(X)
H = model.components_


In [ ]:
X_new = np.array([[1, 0], [1, 6.1], [1, 0], [1, 4], [3.2, 1], [0, 4]])
W_new = model.transform(X_new)

In [ ]:
W

In [ ]:
H

In [ ]:
X_new

In [ ]:
W_new

Using scikit-learn. We use the built-in 20 newsgroups dataset and see what topics are generated.

In [ ]:
#https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [ ]:
data= fetch_20newsgroups(remove=('headers', 'footers', 'quotes')).data

In [ ]:
# convert the text to a tf-idf weighted term-document matrix
 
vectorizer = TfidfVectorizer(max_features=2000, min_df=10, stop_words='english')
X = vectorizer.fit_transform(data)
idx_to_word = np.array(vectorizer.get_feature_names())
 
# apply NMF
 
nmf = NMF(n_components=20, solver="mu")
W = nmf.fit_transform(X)
H = nmf.components_
 
# print the topics
 
for i, topic in enumerate(H):
    print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in idx_to_word[topic.argsort()[-10:]]])))